# ProtSpace: Interactive protein embedding visualization
### About ProtSpace ([github](https://github.com/tsenoner/protspace))
ProtSpace is a tool for interactive visualization of protein embeddings that:
- Converts high-dimensional protein embeddings into 2D/3D visualizations
- Supports multiple dimension reduction methods (PCA, UMAP, t-SNE, PaCMAP)
- Allows annotation-based coloring and shaping of data points
- Integrates protein structure visualization alongside embedding space
- Enables publication-quality exports and sharing of visualization sessions

### Basic Workflow:
1. Upload protein embeddings (H5 file)
2. Upload feature annotations (CSV file)
3. Choose visualization methods
4. Explore your protein space interactively

In [ ]:
#@title Install Dependencies and Import Libraries (~2min)
%%capture
!pip install protspace

import sys
import os
from google.colab import files
from pathlib import Path

from protspace.app import ProtSpace

## Data preparation

In [ ]:
#@title 📤 Data upload and file analysis system
#@markdown Choose your data source by selecting one of the following options:

import requests
import h5py
import pandas as pd
import os
from io import BytesIO
from IPython.display import clear_output
from google.colab import files
from urllib.parse import urlparse

class DataAnalysisSystem:
    def __init__(self):
        self.h5_allowed_extensions = {'.h5', '.hdf5', '.hdf'}
        self.csv_allowed_extensions = {'.csv'}

    def validate_file_extension(self, filename, allowed_extensions):
        """Check if file has an allowed extension."""
        ext = os.path.splitext(filename)[1].lower()
        if ext not in allowed_extensions:
            print(f"❌ Invalid file type. Please upload one of: {', '.join(allowed_extensions)}")
            return False
        return True

    def save_file(self, content, filename):
        """Save file to current directory."""
        with open(filename, 'wb') as f:
            f.write(content)
        return filename

    def handle_file_upload(self, file_type, allowed_extensions):
        """Upload and validate a file, saving it locally."""
        print(f"\n=== Upload {file_type} ===")
        print(f"Allowed extensions: {', '.join(allowed_extensions)}")

        uploaded = files.upload()
        if not uploaded:
            return None, None

        filename = list(uploaded.keys())[0]
        content = uploaded[filename]

        if not self.validate_file_extension(filename, allowed_extensions):
            return None, None

        # Save the uploaded file locally
        filepath = self.save_file(content, filename)
        print(f"✅ File '{filename}' uploaded and saved")
        return filepath, filename

    def get_github_filename(self, url):
        """Extract original filename from GitHub URL."""
        return os.path.basename(urlparse(url).path)

    def download_and_save_github_file(self, url):
        """Download a file from GitHub and save it with original filename."""
        # Get original filename from GitHub URL
        filename = self.get_github_filename(url)

        # Download the file
        raw_url = url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')
        response = requests.get(raw_url, timeout=30)
        response.raise_for_status()

        # Save with original filename
        filepath = self.save_file(response.content, filename)
        return filepath

    def analyze_data_structure(self, h5_filepath, csv_filepath):
        """Analyze both files and show their structure using local file paths."""
        # Read CSV file directly from disk
        df = pd.read_csv(csv_filepath)

        # Read H5 file directly from disk
        with h5py.File(h5_filepath, 'r') as hdf:
            keys = list(hdf.keys())
            first_embedding = hdf[keys[0]]

            h5_ids = set(keys)
            csv_ids = set(df['identifier'])
            matching_ids = h5_ids.intersection(csv_ids)

            print("\n=== Data Analysis Results ===")
            print(f"\nCSV File ({csv_filepath}):")
            print(f"- Entries: {len(df):,}")
            print(f"- Columns: {', '.join(df.columns)}")

            print(f"\nH5 File ({h5_filepath}):")
            print(f"- Embeddings: {len(keys):,}")
            print(f"- Dimensions: {first_embedding.shape}")
            print(f"- Data type: {first_embedding.dtype}")

            print(f"\nAlignment:")
            print(f"- Matching entries: {len(matching_ids):,}")
            print(f"- Only in CSV: {len(csv_ids - h5_ids):,}")
            print(f"- Only in H5: {len(h5_ids - csv_ids):,}")

data_source = "Use example data from GitHub" #@param ["Use example data from GitHub", "Upload my own files"]

try:
    analyzer = DataAnalysisSystem()

    if data_source == "Use example data from GitHub":
        print("Downloading example files...")

        # GitHub URLs with original filenames
        h5_url = "https://github.com/tsenoner/protspace/blob/main/data/Pla2g2/embs/emb_esm2_3b.h5"
        csv_url = "https://github.com/tsenoner/protspace/blob/main/data/Pla2g2/Pla2g2.csv"

        h5_filepath = analyzer.download_and_save_github_file(h5_url)
        csv_filepath = analyzer.download_and_save_github_file(csv_url)
        print("✅ Downloads complete and saved locally")
    else:
        # Handle H5 file upload
        h5_filepath, h5_filename = None, None
        while h5_filepath is None:
            h5_filepath, h5_filename = analyzer.handle_file_upload("Embedding File", analyzer.h5_allowed_extensions)

        # Handle CSV file upload
        csv_filepath, csv_filename = None, None
        while csv_filepath is None:
            csv_filepath, csv_filename = analyzer.handle_file_upload("Feature File", analyzer.csv_allowed_extensions)

    # Analyze the files using their local paths
    analyzer.analyze_data_structure(h5_filepath, csv_filepath)

except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("Please try again or write a GitHub issue if the problem persists.")

In [ ]:
#@title 📤 Data File Analysis System
#@markdown Choose your data source by selecting one of the following options:

import requests
import h5py
import pandas as pd
import numpy as np
from io import BytesIO
import os
from IPython.display import HTML, display, clear_output
from typing import Tuple, Dict, Set, Optional
from dataclasses import dataclass
from tqdm.notebook import tqdm

@dataclass
class FileValidationResult:
    """Contains the validation results for uploaded files."""
    is_valid: bool
    message: str
    content: Optional[bytes] = None

class DataAnalysisSystem:
    """
    A comprehensive system for analyzing H5 embeddings and CSV feature files.

    This class provides methods for file validation, data structure analysis,
    and alignment checking between H5 and CSV files.
    """

    def __init__(self):
        self.h5_allowed_extensions = {'.h5', '.hdf5', '.hdf'}
        self.csv_allowed_extensions = {'.csv'}

    def download_github_file(self, url: str, description: str) -> bytes:
        """
        Downloads a file from GitHub and returns its content.

        Args:
            url: GitHub URL of the file
            description: Description of the file for progress messages

        Returns:
            bytes: The file content

        Raises:
            Exception: If download fails
        """
        print(f"📥 Downloading {description}...")
        raw_url = url.replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/')

        try:
            response = requests.get(raw_url, timeout=30)
            response.raise_for_status()
            print(f"✅ Successfully downloaded {description}")
            return response.content
        except requests.exceptions.RequestException as e:
            raise Exception(f"❌ Failed to download {description} from {url}: {str(e)}")

    def validate_file_extension(self, filename: str, allowed_extensions: Set[str]) -> Tuple[bool, str]:
        """
        Validates if the file has an allowed extension.

        Args:
            filename: Name of the file to validate
            allowed_extensions: Set of allowed file extensions

        Returns:
            Tuple containing validation result (bool) and message (str)
        """
        ext = os.path.splitext(filename)[1].lower()
        if ext not in allowed_extensions:
            allowed_ext_str = ", ".join(allowed_extensions)
            return False, f"❌ Invalid file type. Please upload a file with one of these extensions: {allowed_ext_str}"
        return True, "✅ File extension is valid"

    def validate_h5_file(self, content: bytes, filename: str) -> FileValidationResult:
        """
        Validates H5 file content and structure.

        Args:
            content: Binary content of the H5 file
            filename: Name of the file

        Returns:
            FileValidationResult object containing validation results
        """
        is_valid, message = self.validate_file_extension(filename, self.h5_allowed_extensions)
        if not is_valid:
            return FileValidationResult(False, message)

        try:
            with h5py.File(BytesIO(content), 'r') as f:
                if len(f.keys()) == 0:
                    return FileValidationResult(False, "❌ The H5 file appears to be empty")

                # Validate embedding dimensions consistency
                first_key = list(f.keys())[0]
                expected_shape = f[first_key].shape

                for key in tqdm(f.keys(), desc="Validating embeddings"):
                    if f[key].shape != expected_shape:
                        return FileValidationResult(False,
                            f"❌ Inconsistent embedding dimensions found at key '{key}'")

            return FileValidationResult(True, "✅ H5 file successfully validated!", content)
        except Exception as e:
            return FileValidationResult(False, f"❌ Error reading H5 file: {str(e)}")

    def validate_csv_file(self, content: bytes, filename: str) -> FileValidationResult:
        """
        Validates CSV file content and structure.

        Args:
            content: Binary content of the CSV file
            filename: Name of the file

        Returns:
            FileValidationResult object containing validation results
        """
        is_valid, message = self.validate_file_extension(filename, self.csv_allowed_extensions)
        if not is_valid:
            return FileValidationResult(False, message)

        try:
            df = pd.read_csv(BytesIO(content))

            # Validate required columns
            if 'identifier' not in df.columns:
                return FileValidationResult(False, "❌ Error: CSV file must contain an 'identifier' column")

            # Check for duplicate identifiers
            duplicates = df['identifier'].duplicated().sum()
            if duplicates > 0:
                return FileValidationResult(False,
                    f"❌ Error: Found {duplicates} duplicate identifier(s) in CSV file")

            # Validate identifier format
            invalid_ids = df['identifier'].isna().sum()
            if invalid_ids > 0:
                return FileValidationResult(False,
                    f"❌ Error: Found {invalid_ids} invalid or missing identifier(s)")

            return FileValidationResult(True, "✅ CSV file successfully validated!", content)
        except Exception as e:
            return FileValidationResult(False, f"❌ Error reading CSV file: {str(e)}")

    def analyze_data_structure(self, h5_content: bytes, csv_content: bytes) -> Dict:
        """
        Performs comprehensive analysis of both files' structure and alignment.

        Args:
            h5_content: Binary content of the H5 file
            csv_content: Binary content of the CSV file

        Returns:
            Dictionary containing analysis results
        """
        analysis_results = {}

        # Analyze CSV structure
        features_df = pd.read_csv(BytesIO(csv_content))
        analysis_results['csv'] = {
            'total_entries': len(features_df),
            'columns': features_df.columns.tolist(),
            'sample_data': features_df.head(1).to_dict('records')[0]
        }

        # Analyze HDF5 structure
        with h5py.File(BytesIO(h5_content), 'r') as hdf:
            embeddings_keys = list(hdf.keys())
            first_key = embeddings_keys[0]
            example_embedding = hdf[first_key]

            analysis_results['h5'] = {
                'total_embeddings': len(embeddings_keys),
                'embedding_dimensions': example_embedding.shape,
                'embedding_dtype': str(example_embedding.dtype),
                'memory_usage': example_embedding.nbytes * len(embeddings_keys) / (1024 * 1024)  # MB
            }

            # File alignment analysis
            embeddings_set = set(embeddings_keys)
            identifiers_set = set(features_df['identifier'])

            analysis_results['alignment'] = {
                'missing_in_h5': list(identifiers_set - embeddings_set),
                'missing_in_csv': list(embeddings_set - identifiers_set),
                'total_aligned': len(embeddings_set.intersection(identifiers_set))
            }

        self._display_analysis_results(analysis_results)
        return analysis_results

    def _display_analysis_results(self, results: Dict):
        """
        Displays analysis results in a formatted manner.

        Args:
            results: Dictionary containing analysis results
        """
        print("\n📊 === Data Structure Analysis ===")

        print("\n📋 Feature File Structure:")
        print(f"• Total entries: {results['csv']['total_entries']:,}")
        print(f"• Columns: {', '.join(results['csv']['columns'])}")

        print("\n📦 Embedding File Structure:")
        print(f"• Total embeddings: {results['h5']['total_embeddings']:,}")
        print(f"• Embedding dimensions: {results['h5']['embedding_dimensions']}")
        print(f"• Data type: {results['h5']['embedding_dtype']}")
        print(f"• Approximate memory usage: {results['h5']['memory_usage']:.2f} MB")

        print("\n🔍 Alignment Analysis:")
        print(f"• Successfully aligned entries: {results['alignment']['total_aligned']:,}")
        print(f"• Entries in CSV but not in H5: {len(results['alignment']['missing_in_h5']):,}")
        print(f"• Entries in H5 but not in CSV: {len(results['alignment']['missing_in_csv']):,}")

        if results['alignment']['missing_in_h5'] or results['alignment']['missing_in_csv']:
            print("\n⚠️ Warning: Misaligned entries detected. This may affect analysis results.")

data_source = "Upload my own files" #@param ["Use example data from GitHub", "Upload my own files"]

try:
    analyzer = DataAnalysisSystem()

    if data_source == "Use example data from GitHub":
        h5_url = "https://github.com/tsenoner/protspace/blob/main/data/Pla2g2/embs/emb_esm2_3b.h5"
        csv_url = "https://github.com/tsenoner/protspace/blob/main/data/Pla2g2/Pla2g2.csv"

        h5_content = analyzer.download_github_file(h5_url, "embedding file (H5)")
        csv_content = analyzer.download_github_file(csv_url, "feature file (CSV)")
    else:
        print("\n📤 === File Upload System ===")

        # Step 1: H5 file upload
        while True:
            print("\n=== Step 1: Upload Embedding File ===")
            print("Please upload a file with one of these extensions: .h5, .hdf5, .hdf")

            uploaded = files.upload()
            if not uploaded:
                continue

            filename = list(uploaded.keys())[0]
            result = analyzer.validate_h5_file(uploaded[filename], filename)
            print(result.message)

            if result.is_valid:
                h5_content = result.content
                break

            clear_output(wait=True)

        # Step 2: CSV file upload
        while True:
            print("\n=== Step 2: Upload Feature File ===")
            print("Please upload a CSV file containing the identifiers")

            uploaded = files.upload()
            if not uploaded:
                continue

            filename = list(uploaded.keys())[0]
            result = analyzer.validate_csv_file(uploaded[filename], filename)
            print(result.message)

            if result.is_valid:
                csv_content = result.content
                break

            clear_output(wait=True)

    # Perform analysis
    analyzer.analyze_data_structure(h5_content, csv_content)

except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    print("Please try again or contact support if the problem persists.")

In [ ]:
#@title 📂 File Paths
#@markdown ### Enter paths to your files

#@markdown #### Path to embedding file (.h5/.hdf5/.hdf):
embedding_file = "emb_esm2_3b.h5" #@param {type:"string"}

#@markdown #### Path to feature file (.csv):
feature_file = "Pla2g2.csv" #@param {type:"string"}

# Validate files
valid_files = True
if not embedding_file:
    print("⚠️ Please enter path to embedding file")
    valid_files = False
elif not os.path.exists(embedding_file):
    print(f"⚠️ Embedding file not found: {embedding_file}")
    valid_files = False
elif not any(embedding_file.endswith(ext) for ext in ['.h5', '.hdf5', '.hdf']):
    print(f"⚠️ Embedding file must be .h5, .hdf5, or .hdf format")
    valid_files = False

if not feature_file:
    print("⚠️ Please enter path to feature file")
    valid_files = False
elif not os.path.exists(feature_file):
    print(f"⚠️ Feature file not found: {feature_file}")
    valid_files = False
elif not feature_file.endswith('.csv'):
    print(f"⚠️ Feature file must be .csv format")
    valid_files = False

if valid_files:
    print("✅ Files validated successfully")
    output_file = str(Path(embedding_file).with_suffix('.json'))

In [ ]:
#@title 🔧 Configure Visualization
#@markdown ### Choose dimension reduction methods:
#@markdown - **PCA**: Fast, linear reduction
#@markdown - **UMAP**: Preserves global and local structure
#@markdown - **t-SNE**: Focuses on local structure
#@markdown - **PaCMAP**: Balances global and local structure

use_pca = True #@param {type:"boolean"}
use_umap = False #@param {type:"boolean"}
use_tsne = False #@param {type:"boolean"}
use_pacmap = False #@param {type:"boolean"}

#@markdown ### Choose dimensions:
dimensions = "2D only" #@param ["2D only", "3D only", "2D and 3D"]

# Build methods string
methods = []

def add_method(use_method, method_name):
    if use_method:
        if dimensions == "2D only":
            methods.append(f"{method_name}2")
        elif dimensions == "3D only":
            methods.append(f"{method_name}3")
        else:
            methods.extend([f"{method_name}2", f"{method_name}3"])

add_method(use_pca, "pca")
add_method(use_umap, "umap")
add_method(use_tsne, "tsne")
add_method(use_pacmap, "pacmap")

methods_str = " ".join(methods)
params_str = ""

## Advanced settings (optional)

In [ ]:
#@title ## Advanced Parameters

#@markdown #### UMAP Parameters:
if use_umap:
    #@markdown - Number of neighbors influences locality preservation
    umap_n_neighbors = 50 #@param {type:"slider", min:2, max:200, step:1}
    #@markdown - Minimum distance between points
    umap_min_dist = 0.5 #@param {type:"slider", min:0.0, max:1.0, step:0.01}
    #@markdown - Distance metric
    umap_metric = "euclidean" #@param ["euclidean", "cosine"]

# @markdown ---
#@markdown #### t-SNE Parameters:
if use_tsne:
    #@markdown - Perplexity balances local and global structure
    tsne_perplexity = 30 #@param {type:"slider", min:5, max:100, step:5}
    #@markdown - Learning rate influences optimization
    tsne_learning_rate = 200 #@param {type:"number"}

# @markdown ---
#@markdown #### PaCMAP Parameters:
if use_pacmap:
    #@markdown - Number of neighbors
    pacmap_n_neighbors = 25 #@param {type:"slider", min:2, max:100, step:1}
    #@markdown - MN ratio (Mid-Near pairs ratio): Controls local structure preservation
    #@markdown   - Higher values (→1.0): Better preserves local structure
    #@markdown   - Lower values (→0.1): Allows more global structure influence
    pacmap_mn_ratio = 0.5 #@param {type:"slider", min:0.1, max:1.0, step:0.1}
    #@markdown - FP ratio (Further Pairs ratio): Controls global structure preservation
    #@markdown   - Higher values (→5.0): Better preserves global structure, more separation between clusters
    #@markdown   - Lower values (→0.1): Focuses more on local relationships
    pacmap_fp_ratio = 2.0 #@param {type:"slider", min:0.1, max:5.0, step:0.1}
    #@markdown
    #@markdown Recommended combinations:
    #@markdown - Balanced view: MN=0.5, FP=2.0
    #@markdown - Local focus: MN=0.8, FP=1.0
    #@markdown - Global focus: MN=0.3, FP=3.0

# Build parameter string
params = []

if use_umap:
    params.extend([
        f"--n_neighbors {umap_n_neighbors}",
        f"--min_dist {umap_min_dist}",
        f"--metric {umap_metric}"
    ])

if use_tsne:
    params.extend([
        f"--perplexity {tsne_perplexity}",
        f"--learning_rate {tsne_learning_rate}"
    ])

if use_pacmap:
    params.extend([
        f"--n_neighbors {pacmap_n_neighbors}",
        f"--mn_ratio {pacmap_mn_ratio}",
        f"--fp_ratio {pacmap_fp_ratio}"
    ])

params_str = " ".join(params)

## 📊 Generate JSON File

In [ ]:
#@title Generate the visualization data file

if not valid_files:
    print("⚠️ Please fix file path issues before continuing")
else:
    print(f"Generating visualization data...")
    !protspace-json -i {embedding_file} -m {feature_file} -o {output_file} --methods {methods_str} {params_str}
    print(f"✅ JSON file saved as: {output_file}")

## 🚀 Launch ProtSpace

In [ ]:
#@title 🌟 Visualization Settings and Launch {display-mode: "form"}
#@markdown ### Configure and launch ProtSpace visualization

#@markdown #### Path to JSON file:
json_file = "" #@param {type:"string"}

#@markdown #### Visualization Height (pixels):
jupyter_height = 800 #@param {type:"slider", min:400, max:1200, step:50}

#@markdown #### Jupyter Display Mode:
jupyter_mode = "inline" #@param ["inline", "external"]

import os
import sys
from IPython.display import display, HTML, clear_output

def show_launch_info(json_file, height, mode):
    """Display launch information in a formatted box separate from the dashboard."""
    info_html = f"""
    <div style="background-color: #666666; padding: 10px; margin-bottom: 20px; border-radius: 5px;">
        <p style="margin: 0;"><strong>📊 Launching ProtSpace with:</strong></p>
        <ul style="margin: 10px 0;">
            <li>File: {os.path.basename(json_file)}</li>
            <li>Height: {height}px</li>
            <li>Mode: {mode}</li>
        </ul>
    </div>
    """
    display(HTML(info_html))

def launch_protspace(json_file, height=800, mode="inline"):
    """
    Launch ProtSpace with customized visualization settings.

    Args:
        json_file: Path to the JSON configuration file
        height: Height of the visualization in pixels
        mode: Jupyter display mode ('inline', 'external', 'jupyterlab', 'tab')
    """
    # Use generated JSON file if no other file specified
    if not json_file and 'output_file' in globals():
        json_file = output_file

    if not json_file:
        display(HTML(
            '<div style="color: #dc3545; padding: 10px;">'
            '⚠️ No JSON file specified. Please enter a file path or ensure a default output file exists.'
            '</div>'
        ))
        return False

    if not os.path.exists(json_file):
        display(HTML(
            '<div style="color: #dc3545; padding: 10px;">'
            f'⚠️ JSON file not found: {json_file}'
            '</div>'
        ))
        return False

    # Clear any previous output
    clear_output(wait=True)

    # Show launch information
    show_launch_info(json_file, height, mode)

    original_stdout, original_stderr = sys.stdout, sys.stderr
    try:
        sys.stdout = sys.stderr = open(os.devnull, 'w')
        protspace = ProtSpace(default_json_file=json_file)
        app = protspace.create_app()
        app.run(
            jupyter_mode=mode,
            jupyter_height=height,  # For inline mode, apply height restriction
            jupyter_width='100%',
            dev_tools_silence_routes_logging=True,
            dev_tools_prune_errors=True
        )
        return True
    except Exception as e:
        display(HTML(
            '<div style="color: #dc3545; padding: 10px;">'
            f'❌ Error launching ProtSpace: {str(e)}'
            '</div>'
        ))
        return False
    finally:
        sys.stdout, sys.stderr = original_stdout, original_stderr

# Launch ProtSpace with the configured settings
launch_protspace(json_file, height=jupyter_height, mode=jupyter_mode)